### Uso de Red Neuronal para trabajar Series de Tiempo

In [ ]:
from math import sqrt
from numpy import array
from numpy import mean
from numpy import std
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from matplotlib import pyplot
%matplotlib inline  
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Funciones

In [ ]:
# split a univariate dataset into train/test sets
def division_entreno_prueba(datos, n_prueba):
    return datos[:-n_prueba], datos[-n_prueba:]

Para la entrada de datos, a la red, es necesario transformarlos de una lista o serie, a un formato adecuado para el aprendizaje supervisado

In [ ]:
def series_a_supervisado(datos, n_entrada, n_salida = 1):
	df = DataFrame(datos)
	cols = []
	# secuencia de entrada (t-n, ... t-1)
	for i in range(n_entrada, 0, -1):
		cols.append(df.shift(i))
	# secuencia predicha (t, t+1, ... t+n)
	for i in range(0, n_salida):
		cols.append(df.shift(-i))
	# juntarlo todo
	agg = concat(cols, axis = 1)
	# botar columnas con valores NaN
	agg.dropna(inplace = True)
	return agg.values

Obtener el valor de la raiz cuadrada de la media de los errores al cuadrado RMSE

In [ ]:
def medir_rmse(actual, predicho):
	return sqrt(mean_squared_error(actual, predicho))

Ajustar los datos al modelo

In [ ]:
def ajustar_modelo(entreno, config):
	# desempacar la configuración
	n_entrada, n_nodos, n_epocas, n_tandas = config
    #[24, 500, 100, 100]
    # preparar los datos
	datos = series_a_supervisado(entreno, n_entrada)
	entreno_x, entreno_y = datos[:, :-1], datos[:, -1]
	# definir el modelo
	modelo = tf.keras.Sequential()
	modelo.add(tf.keras.layers.Dense(n_nodos, activation='relu', input_dim = n_entrada))
	modelo.add(tf.keras.layers.Dense(1))
	modelo.compile(loss='mse', optimizer='adam')
	# ajustar
	modelo.fit(entreno_x, entreno_y, epochs = n_epocas, batch_size = n_tandas, verbose = 0)
	return modelo

Predecir con un modelo ya ajustado

In [ ]:
def prediccion_modelo(modelo, historia, config):
	# desempacar la configuración
	n_entrada, _, _, _ = config
	# preparar los datos
	x_entrada = array(historia[-n_entrada:]).reshape(1, n_entrada)
	# predecir
	yhat = modelo.predict(x_entrada, verbose = 0)
	return yhat[0]

Validación hacia el frente para datos unviariados

In [ ]:
def validacion_al_frente(datos, n_prueba, cfg):
    predicciones = []
    # dividir el conjunto de datos
    entreno, prueba = division_entreno_prueba(datos, n_prueba)
    # ajustar el modelo
    modelo = ajustar_modelo(entreno, cfg)
    # grabar la historia con el conjunto de datos de entrenamiento
    historia = [x for x in entreno]
    # pasar por cada incremento de tiempo en el conjunto de prueba
    for i in range(len(prueba)):
        # ajustar el modelo a los datos y predecir los datos históricos
        yhat = prediccion_modelo(modelo, historia, cfg)
        # agregar el dato predicho en la lista de preducciones
        predicciones.append(yhat)
        # agregar la observación a la historia para la siguiente iteración
        historia.append(prueba[i])
    # estimar el error de las predicciones
    error = medir_rmse(prueba, predicciones)
    print(f' > {error:.3f}')
    return [error, predicciones]

Evaluación iterativa de una configuraciñon

In [ ]:
 def evaluacion_iter(datos, config, n_prueba, n_repeticiones = 5):
	# ajustar y evaluar el modelo n veces
    resultados = []
    predicciones = []
    for _ in range(n_repeticiones):
        resultados_temp, predicciones_temp = validacion_al_frente(datos, n_prueba, config)
        resultados.append(resultados_temp)
        predicciones.append(predicciones_temp)
    return [resultados, predicciones]

Resumir el rendimiento del modelo

In [ ]:
def resumir_resultados(nombre, resultados):
	# imprimir un resúmen
	media_result, std_result = mean(resultados), std(resultados)
	print(f'{nombre}: {media_result:.3f} RMSE (+/- {std_result:.3f})')
	# gráficas de caja y bigotes
	pyplot.boxplot(resultados)
	pyplot.show()

Encontrar la mejor solución

In [ ]:
def buscar_mejor_solucion(series, nombre, metodo = "red neuronal"):
    datos = series.values
    n_prueba = int(len(datos)*0.2) # porcentaje utilizado para prueba
    config = [24, 500, 100, 100]
    resultados, predicciones = evaluacion_iter(datos, config, n_prueba)
    resumir_resultados('mlp', resultados)
    plt.savefig("resultados_{nombre}_{metodo}.png")
    plt.show()
    entreno, prueba = division_entreno_prueba(datos,n_prueba)
    prediccion = pd.DataFrame(list(entreno.flatten()) + np.array(predicciones[0]).flatten().tolist())
    ax = pd.DataFrame(datos).plot(label="Original") # datos originales
    prediccion.plot(ax=ax, alpha=.7, figsize=(14,7))
    plt.savefig("pred_{nombre}_{metodo}.png")
    plt.show()

# Nacimiento Niñas

In [ ]:
series = pd.read_csv('daily-total-female-births.csv', header=0,index_col=0)
buscar_mejor_solucion(series,"nacimientos")

# Ventas Shampoo

In [ ]:
series = pd.read_csv('shampoo.csv', header=0, index_col=0)
buscar_mejor_solucion(series,"shampoo")

# Temperaturas

In [ ]:
series = pd.read_csv('monthly-mean-temp.csv', header=0,parse_dates=["Month"],index_col=0)
buscar_mejor_solucion(series,"temp")

# Venta de carros

In [ ]:
series = read_csv('monthly-car-sales.csv', header=0, index_col=0)
buscar_mejor_solucion(series,"carros")

### Podríamos tratar de usar otra configuruación de entrada, por ejemplo un año (12 meses)

In [ ]:
series = read_csv('monthly-car-sales.csv', header=0, index_col=0)
datos = series.values
# división de datos
n_prueba = int(len(datos) * 0.2) # porcentaje utilizao para prueba


# definir config
#config = [24, 500, 100, 100]
config = [[24, 500, 100, 100],[12, 500, 100, 100]]

# búsqueda en malla
resultados, predicciones = evaluacion_iter(datos, config[1], n_prueba)

# resumir resultados
resumir_resultados('mlp', resultados)

# Mostrar Predicciones

In [ ]:
entreno, prueba = division_entreno_prueba(datos,n_prueba)
prediccion = pd.DataFrame(list(entreno.flatten())+np.array(predicciones[0]).flatten().tolist())
ax = pd.DataFrame(datos).plot(label="Original") # datos originales
prediccion.plot(ax=ax, alpha=.7, figsize=(14,7))
plt.show()